# API Examples - Quick Reference

Condensed notebook covering the core Platform endpoints (updated 18 DEC 2025):

### CoreMachine API (Direct ETL Access)
1. Health Check
2. Container Check (Sync)
3. Process Vector
4. **Process Raster v2** (single file, ≤800 MB)
5. **Process Large Raster v2** (100 MB - 30 GB, tiled processing)
6. **Process Raster Collection v2** (≤20 files, each ≤800 MB)
7. Rejection Examples (size/count limit violations)

### Platform API (Anti-Corruption Layer)
8. Platform Single Raster (DDH identifiers)
9. Platform Raster Collection (DDH identifiers)
10. Platform Status Check
11. **Unpublish Vector** (3 options: DDH IDs, request_id, cleanup mode)
12. **Unpublish Raster** (3 options: DDH IDs, request_id, cleanup mode)
13. Platform Operations (health, stats, failures)

### Size Routing Summary

| File Size | Job Type | Notes |
|-----------|----------|-------|
| ≤800 MB | `process_raster_v2` | Standard COG conversion |
| 100 MB - 30 GB | `process_large_raster_v2` | Tiled COG workflow |
| Collection ≤20 files | `process_raster_collection_v2` | Each file must be ≤800 MB |

## Setup

In [ ]:
import requests
import json
import time

# =============================================================================
# CONFIGURATION - All variables defined here
# =============================================================================

# Function App Base URL
BASE_URL = "https://rmhazuregeoapi-a3dma3ctfdgngwf6.eastus-01.azurewebsites.net"

# Storage Containers
BRONZE_CONTAINER = "rmhazuregeobronze"  # Main bronze container for raw input
SILVER_CONTAINER = "silver-cogs"         # Processed COGs output

# PostGIS Schema
POSTGIS_SCHEMA = "geo"

# =============================================================================
# Helper Functions
# =============================================================================

def api_call(method, endpoint, data=None, params=None, timeout=30):
    """Make API call and return formatted response."""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    
    print(f"\n{'='*60}")
    print(f"{method} {endpoint}")
    print(f"{'='*60}")
    
    if data:
        print(f"\nRequest Body:")
        print(json.dumps(data, indent=2))
    
    try:
        if method == "GET":
            response = requests.get(url, params=params, timeout=timeout)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print(f"\nStatus: {response.status_code}")
        
        try:
            result = response.json()
            print(f"\nResponse:")
            print(json.dumps(result, indent=2, default=str))
            return result
        except:
            print(f"\nResponse (text): {response.text[:500]}")
            return response.text
            
    except requests.exceptions.Timeout:
        print(f"\n❌ Request timed out (timeout={timeout}s)")
        return None
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

def check_job_status(job_id, max_polls=20, poll_interval=5):
    """Poll job status until completion or timeout."""
    print(f"\n{'='*60}")
    print(f"Polling job: {job_id}")
    print(f"{'='*60}")
    
    for i in range(max_polls):
        result = requests.get(f"{BASE_URL}/api/jobs/status/{job_id}", timeout=30).json()
        status = result.get("status", "unknown")
        stage = result.get("current_stage", "?")
        
        print(f"  [{i+1}/{max_polls}] Status: {status}, Stage: {stage}")
        
        if status in ["completed", "failed"]:
            print(f"\nFinal Result:")
            print(json.dumps(result, indent=2, default=str))
            return result
        
        time.sleep(poll_interval)
    
    print(f"\n⚠️ Polling timeout after {max_polls * poll_interval}s")
    return result

# Display configuration
print("=" * 60)
print("API CONFIGURATION")
print("=" * 60)
print(f"Base URL:          {BASE_URL}")
print(f"Bronze Container:  {BRONZE_CONTAINER}")
print(f"Silver Container:  {SILVER_CONTAINER}")
print(f"PostGIS Schema:    {POSTGIS_SCHEMA}")
print("=" * 60)

---
## 1. Health Check

Comprehensive system health check (~60s due to database, Service Bus, and storage checks).

In [ ]:
# Health Check (takes ~60s)
result = api_call("GET", "/api/health", timeout=90)

---
## 2. Container Check (Sync)

Quick synchronous endpoint to list blobs in a container. No job queue - returns immediately.

**Parameters:**
- `suffix`: Filter by extension (e.g., `.tif`, `.geojson`)
- `metadata`: `true` (default) returns full blob info, `false` returns just names
- `limit`: Max blobs to return (default: 500, max: 10000)

In [ ]:
# Container Check - Sync endpoint (returns immediately, no job queue)
# List first 10 TIF files with full metadata

result = api_call("GET", f"/api/containers/{BRONZE_CONTAINER}/blobs", 
                  params={"suffix": ".tif", "limit": 10, "metadata": "true"})

# Show summary
if result and isinstance(result, dict):
    count = result.get("count", 0)
    blobs = result.get("blobs", [])
    print(f"\n📊 Found {count} TIF files")
    if blobs:
        total_mb = sum(b.get("size_mb", 0) for b in blobs)
        print(f"📦 Total size: {total_mb:.2f} MB")

---
## 3. Process Vector

Submit a vector file (GeoJSON, Shapefile, GeoPackage) for ingestion into PostGIS.

In [ ]:
# Submit Vector
vector_request = {
    "dataset_id": "test-vectors",
    "resource_id": "geojson-8",
    "version_id": "v1",
    "container_name": BRONZE_CONTAINER,
    "file_name": "8.geojson",
    "service_name": "Test GeoJSON 8"
}

result = api_call("POST", "/api/platform/submit", vector_request)
vector_job_id = result.get("job_id") if result else None
print(f"\n📋 Job ID: {vector_job_id}")

In [ ]:
# Check Vector Job Status
if vector_job_id:
    check_job_status(vector_job_id)
else:
    print("⚠️ No job_id from previous cell")

---
## 4. Process Raster (Single File)

Submit a single raster file for COG conversion and STAC cataloging.

### Size Limits (13 DEC 2025)

| Limit | Value | Behavior |
|-------|-------|----------|
| **Max file size** | 800 MB | Files >800MB rejected → use `process_large_raster_v2` |
| **Min file size** | None | Any size accepted |

**Pre-flight validation** automatically checks file size before processing.

### Test Data
- **dctest.tif** (25.8 MB) - Small RGB GeoTIFF, processes in ~22 seconds
- **antigua.tif** (11.16 GB) - Too large, will be rejected with error message

In [ ]:
# Submit Single Raster via CoreMachine API (direct)
# Using dctest.tif (25.8 MB) - verified working 13 DEC 2025

raster_request = {
    "blob_name": "dctest.tif",
    "container_name": BRONZE_CONTAINER
}

result = api_call("POST", "/api/jobs/submit/process_raster_v2", raster_request)
raster_job_id = result.get("job_id") if result else None

# Show size metadata from pre-flight validation
if result and "parameters" in result:
    params = result["parameters"]
    print(f"\n📏 Pre-flight Size Check:")
    print(f"   File size: {params.get('_blob_size_mb', 'N/A'):.2f} MB")
    print(f"   File exists: ✅")

print(f"\n📋 Job ID: {raster_job_id}")

In [ ]:
# Check Raster Job Status
if raster_job_id:
    check_job_status(raster_job_id)
else:
    print("⚠️ No job_id from previous cell")

---
## 5. Process Large Raster (100 MB - 30 GB)

Submit a large raster for tiled COG processing. Uses 5-stage workflow:
1. Generate tiling scheme
2. Extract tiles (sequential)
3. Create COGs (parallel)
4. Create MosaicJSON
5. Create STAC collection

### Size Limits (13 DEC 2025)

| Limit | Value | Behavior |
|-------|-------|----------|
| **Min file size** | 100 MB | Files <100MB should use `process_raster_v2` |
| **Max file size** | 30 GB | Files >30GB not supported |

### Test Data
- **antigua.tif** (11.16 GB) - Maxar Vivid Imagery of Antigua island

In [ ]:
# Submit Large Raster via CoreMachine API (direct)
# Using antigua.tif (11.16 GB) - verified 13 DEC 2025
# Note: This is a long-running job (~30+ minutes for tiling and COG creation)

large_raster_request = {
    "blob_name": "antigua.tif",
    "container_name": BRONZE_CONTAINER
}

result = api_call("POST", "/api/jobs/submit/process_large_raster_v2", large_raster_request)
large_raster_job_id = result.get("job_id") if result else None

# Show size metadata from pre-flight validation
if result and "parameters" in result:
    params = result["parameters"]
    size_mb = params.get('_blob_size_mb', 0)
    print(f"\n📏 Pre-flight Size Check:")
    print(f"   File size: {size_mb:.2f} MB ({size_mb/1024:.2f} GB)")
    print(f"   Valid for large raster: {'✅' if 100 <= size_mb <= 30000 else '❌'}")

print(f"\n📋 Job ID: {large_raster_job_id}")

In [ ]:
# Check Large Raster Job Status
if large_raster_job_id:
    check_job_status(large_raster_job_id, max_polls=30, poll_interval=10)  # Longer timeout for large files
else:
    print("⚠️ No job_id from previous cell")

---
## 6. Process Raster Collection (Multi-File)

Submit multiple raster files to be processed as a collection with MosaicJSON.

### Size and Count Limits (13 DEC 2025)

| Limit | Value | Behavior |
|-------|-------|----------|
| **Max files per collection** | 20 | Collections with >20 files rejected |
| **Max individual file size** | 800 MB | Collections with ANY file >800MB rejected |
| **Min files per collection** | 2 | Single files should use `process_raster_v2` |

**Pre-flight validation order:**
1. **Collection count** - Rejected immediately if >20 files (before any blob checks)
2. **Individual file sizes** - Each blob checked in parallel; rejected if ANY exceeds 800MB
3. **File existence** - All blobs must exist in the container

### Size Metadata Captured
After validation, these fields are available in job parameters:
- `_blob_list_count` - Number of files
- `_blob_list_max_size_mb` - Largest file size
- `_blob_list_total_size_mb` - Total size of all files
- `_blob_list_largest_blob` - Name of largest file
- `_blob_list_has_large_raster` - True if any file >800MB

### Test Data
- **namangan/** folder (4 tiles, 1.6 GB total):
  - R1C1: 778 MB, R1C2: 704 MB, R2C1: 73 MB, R2C2: 65 MB
  - All under 800 MB limit ✅

In [ ]:
# Submit Raster Collection via CoreMachine API (direct)
# Using namangan 4-tile collection (1.6 GB total) - verified 13 DEC 2025

collection_request = {
    "container_name": BRONZE_CONTAINER,
    "blob_list": [
        "namangan/namangan14aug2019_R1C1cog.tif",  # 778 MB
        "namangan/namangan14aug2019_R1C2cog.tif",  # 704 MB
        "namangan/namangan14aug2019_R2C1cog.tif",  # 73 MB
        "namangan/namangan14aug2019_R2C2cog.tif"   # 65 MB
    ],
    "collection_id": "namangan-test"
}

result = api_call("POST", "/api/jobs/submit/process_raster_collection_v2", collection_request)
collection_job_id = result.get("job_id") if result else None

# Show size metadata from pre-flight validation
if result and "parameters" in result:
    params = result["parameters"]
    print(f"\n📏 Pre-flight Size Check:")
    print(f"   Files in collection: {params.get('_blob_list_count', 'N/A')}")
    print(f"   Largest file: {params.get('_blob_list_max_size_mb', 0):.2f} MB")
    print(f"   Total size: {params.get('_blob_list_total_size_mb', 0):.2f} MB")
    print(f"   Largest blob: {params.get('_blob_list_largest_blob', 'N/A')}")
    print(f"   Has large raster (>800MB): {'❌ Yes' if params.get('_blob_list_has_large_raster') else '✅ No'}")

print(f"\n📋 Job ID: {collection_job_id}")

In [ ]:
# Check Raster Collection Job Status
if collection_job_id:
    check_job_status(collection_job_id, max_polls=30, poll_interval=10)  # Longer timeout for multi-file
else:
    print("⚠️ No job_id from previous cell")

---
## Quick Reference: Manual Job Status Check

Use this cell to check any job by ID.

In [ ]:
# Manual Job Status Check
# Replace with your job_id
manual_job_id = "YOUR_JOB_ID_HERE"

if manual_job_id != "YOUR_JOB_ID_HERE":
    check_job_status(manual_job_id)
else:
    print("⚠️ Replace 'YOUR_JOB_ID_HERE' with an actual job_id")

---
## 7. Rejection Examples (Size/Count Limit Violations)

These examples demonstrate the pre-flight validation rejecting invalid requests.

In [ ]:
# Example 1: Single raster too large (>800 MB)
# antigua.tif is 11.16 GB - should be rejected with message to use process_large_raster_v2

print("=" * 60)
print("TEST 1: Single raster exceeding 800 MB limit")
print("=" * 60)

large_single_request = {
    "blob_name": "antigua.tif",  # 11.16 GB
    "container_name": BRONZE_CONTAINER
}

result = api_call("POST", "/api/jobs/submit/process_raster_v2", large_single_request)
if result and "error" in result:
    print(f"\n✅ Correctly rejected: {result.get('message', '')[:100]}...")

In [ ]:
# Example 2: Collection with too many files (>20)
# Should be rejected before any blob checks are made

print("=" * 60)
print("TEST 2: Collection exceeding 20 file limit")
print("=" * 60)

too_many_files_request = {
    "container_name": BRONZE_CONTAINER,
    "blob_list": [f"file{i}.tif" for i in range(21)],  # 21 files
    "collection_id": "test-too-many"
}

result = api_call("POST", "/api/jobs/submit/process_raster_collection_v2", too_many_files_request)
if result and "error" in result:
    print(f"\n✅ Correctly rejected: {result.get('message', '')[:100]}...")

In [ ]:
# Example 3: Collection with missing blob
# Should be rejected with list of missing files

print("=" * 60)
print("TEST 3: Collection with non-existent file")
print("=" * 60)

missing_blob_request = {
    "container_name": BRONZE_CONTAINER,
    "blob_list": [
        "namangan/namangan14aug2019_R1C1cog.tif",  # exists
        "nonexistent_file_xyz123.tif"               # does not exist
    ],
    "collection_id": "test-missing"
}

result = api_call("POST", "/api/jobs/submit/process_raster_collection_v2", missing_blob_request)
if result and "error" in result:
    print(f"\n✅ Correctly rejected: {result.get('message', '')[:100]}...")

---
# Platform API (Anti-Corruption Layer)

The Platform API is an **Anti-Corruption Layer (ACL)** that shields external applications from CoreMachine internals. External apps use high-level DDH identifiers (`dataset_id`, `resource_id`, `version_id`); Platform translates them to CoreMachine job parameters automatically.

### Platform API vs CoreMachine API

| Aspect | Platform API | CoreMachine API |
|--------|--------------|-----------------|
| **Audience** | External applications (DDH) | Internal tools, power users |
| **Identifiers** | `dataset_id`, `resource_id`, `version_id` | `blob_name`, `table_name`, `collection_id` |
| **Output naming** | Auto-generated from DDH IDs | You specify everything |
| **Status tracking** | `request_id` (DDH-friendly) | `job_id` (internal hash) |

### Endpoints Summary

| Endpoint | Purpose |
|----------|---------|
| `/api/platform/raster` | Single raster file processing |
| `/api/platform/raster-collection` | Multiple raster files (2-20 files) |
| `/api/platform/submit` | Generic submission (auto-detects data type) |
| `/api/platform/status/{request_id}` | Check request/job status |
| `/api/platform/unpublish/vector` | Remove vector data |
| `/api/platform/unpublish/raster` | Remove raster data |
| `/api/platform/health` | Platform health check |
| `/api/platform/stats` | Aggregated job statistics |
| `/api/platform/failures` | Recent failures |

---
## 8. Platform API - Single Raster

Submit a single raster file using DDH identifiers. Output paths are auto-generated from identifiers.

### Key Benefit
Files exceeding 800 MB are automatically routed to the large raster tiling workflow - no action required.

In [ ]:
# Platform API - Single Raster with DDH Identifiers
# Uses dataset_id/resource_id/version_id for output naming

platform_raster_request = {
    "dataset_id": "test-raster-notebook",
    "resource_id": "dctest",
    "version_id": "v1",
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif",
    "service_name": "DC Test Imagery",
    "access_level": "OUO",
    "description": "Test raster via Platform API"
}

result = api_call("POST", "/api/platform/raster", platform_raster_request)
platform_raster_request_id = result.get("request_id") if result else None
platform_raster_job_id = result.get("job_id") if result else None

print(f"\n📋 Request ID: {platform_raster_request_id}")
print(f"📋 Job ID: {platform_raster_job_id}")
if result:
    print(f"📍 Monitor URL: {result.get('monitor_url', 'N/A')}")

---
## 9. Platform API - Raster Collection

Submit multiple raster files using DDH identifiers. Creates a unified STAC collection with MosaicJSON.

In [ ]:
# Platform API - Raster Collection with DDH Identifiers
# Uses dataset_id/resource_id/version_id for output naming

platform_collection_request = {
    "dataset_id": "namangan-imagery",
    "resource_id": "aug2019",
    "version_id": "v1",
    "container_name": BRONZE_CONTAINER,
    "file_name": [
        "namangan/namangan14aug2019_R1C1cog.tif",
        "namangan/namangan14aug2019_R1C2cog.tif",
        "namangan/namangan14aug2019_R2C1cog.tif",
        "namangan/namangan14aug2019_R2C2cog.tif"
    ],
    "service_name": "Namangan Satellite Imagery",
    "access_level": "OUO"
}

result = api_call("POST", "/api/platform/raster-collection", platform_collection_request)
platform_collection_request_id = result.get("request_id") if result else None

print(f"\n📋 Request ID: {platform_collection_request_id}")
print(f"📋 File Count: {result.get('file_count', 'N/A')}" if result else "")

---
## 10. Platform Status Check

Check request status using DDH-friendly `request_id` (shorter than `job_id`).

Returns comprehensive status including job progress, stage info, and task summary.

In [ ]:
# Platform Status Check - Use request_id from Platform submission
# Replace with your request_id from earlier Platform API calls

platform_request_id = platform_raster_request_id or "YOUR_REQUEST_ID_HERE"

if platform_request_id and platform_request_id != "YOUR_REQUEST_ID_HERE":
    result = api_call("GET", f"/api/platform/status/{platform_request_id}")
    
    if result and result.get("success"):
        print(f"\n📊 Status Summary:")
        print(f"   Job Status: {result.get('job_status', 'N/A')}")
        print(f"   Job Stage: {result.get('job_stage', 'N/A')}")
        print(f"   Data Type: {result.get('data_type', 'N/A')}")
        
        task_summary = result.get('task_summary', {})
        if task_summary:
            print(f"\n   Tasks: {task_summary.get('completed', 0)}/{task_summary.get('total', 0)} completed")
else:
    print("⚠️ Set platform_request_id to a valid request_id")

---
## 11. Unpublish Vector Data

Remove vector data from the platform. Three ways to identify what to delete:

1. **By DDH Identifiers** (Preferred) - Uses `dataset_id`, `resource_id`, `version_id`
2. **By Request ID** - Uses original platform request_id from submission
3. **Cleanup Mode** - Direct table_name for orphaned data

### Workflow Stages
1. **Inventory** - Query `geo.table_metadata` for ETL/STAC linkage
2. **Drop Table** - DROP PostGIS table + DELETE metadata row
3. **Cleanup** - Delete STAC item if linked + create audit record

### Important
- Default `dry_run=true` for safety (shows what would be deleted)
- Set `dry_run=false` to actually execute deletion

In [ ]:
# Unpublish Vector - Option 1: By DDH Identifiers (Preferred)
# dry_run=true shows what would be deleted without executing

unpublish_vector_ddh = {
    "dataset_id": "test-vectors",
    "resource_id": "geojson-8",
    "version_id": "v1",
    "dry_run": True  # Set to False to actually delete
}

result = api_call("POST", "/api/platform/unpublish/vector", unpublish_vector_ddh)
if result:
    print(f"\n📋 Mode: {result.get('mode', 'N/A')}")
    print(f"🔍 Dry Run: {result.get('dry_run', 'N/A')}")
    print(f"📍 Table: {result.get('table_name', 'N/A')}")

In [ ]:
# Unpublish Vector - Option 2: By Request ID
# Uses the request_id from the original platform submission

unpublish_vector_request = {
    "request_id": "YOUR_ORIGINAL_REQUEST_ID",  # From /api/platform/submit response
    "dry_run": True
}

# Uncomment to test (replace with actual request_id)
# result = api_call("POST", "/api/platform/unpublish/vector", unpublish_vector_request)
print("⚠️ Uncomment and replace request_id to test")

In [ ]:
# Unpublish Vector - Option 3: Cleanup Mode (Direct table_name)
# For orphaned tables that don't have platform request records

unpublish_vector_cleanup = {
    "table_name": "orphaned_table_v1_0",
    "schema_name": "geo",  # Optional, defaults to "geo"
    "dry_run": True
}

# Uncomment to test (replace with actual table_name)
# result = api_call("POST", "/api/platform/unpublish/vector", unpublish_vector_cleanup)
print("⚠️ Uncomment and replace table_name to test")

---
## 12. Unpublish Raster Data

Remove raster data from the platform. Three ways to identify what to delete:

1. **By DDH Identifiers** (Preferred) - Uses `dataset_id`, `resource_id`, `version_id`
2. **By Request ID** - Uses original platform request_id from submission
3. **Cleanup Mode** - Direct STAC identifiers for orphaned data

### Workflow Stages
1. **Inventory** - Query STAC item, extract asset hrefs for deletion
2. **Delete Blobs** - Fan-out deletion of COG/MosaicJSON blobs
3. **Cleanup** - Delete STAC item + create audit record

### Important
- Default `dry_run=true` for safety (shows what would be deleted)
- Set `dry_run=false` to actually execute deletion

In [ ]:
# Unpublish Raster - Option 1: By DDH Identifiers (Preferred)
# dry_run=true shows what would be deleted without executing

unpublish_raster_ddh = {
    "dataset_id": "test-raster-notebook",
    "resource_id": "dctest",
    "version_id": "v1",
    "dry_run": True  # Set to False to actually delete
}

result = api_call("POST", "/api/platform/unpublish/raster", unpublish_raster_ddh)
if result:
    print(f"\n📋 Mode: {result.get('mode', 'N/A')}")
    print(f"🔍 Dry Run: {result.get('dry_run', 'N/A')}")
    print(f"📍 STAC Item: {result.get('stac_item_id', 'N/A')}")
    print(f"📁 Collection: {result.get('collection_id', 'N/A')}")

In [ ]:
# Unpublish Raster - Option 2: By Request ID
# Uses the request_id from the original platform submission

unpublish_raster_request = {
    "request_id": "YOUR_ORIGINAL_REQUEST_ID",  # From /api/platform/raster response
    "dry_run": True
}

# Uncomment to test (replace with actual request_id)
# result = api_call("POST", "/api/platform/unpublish/raster", unpublish_raster_request)
print("⚠️ Uncomment and replace request_id to test")

In [ ]:
# Unpublish Raster - Option 3: Cleanup Mode (Direct STAC identifiers)
# For orphaned STAC items that don't have platform request records

unpublish_raster_cleanup = {
    "stac_item_id": "aerial-imagery-2024-site-alpha-v1-0",
    "collection_id": "aerial-imagery-2024",
    "dry_run": True
}

# Uncomment to test (replace with actual STAC identifiers)
# result = api_call("POST", "/api/platform/unpublish/raster", unpublish_raster_cleanup)
print("⚠️ Uncomment and replace STAC identifiers to test")

---
## 13. Platform Operations

Operational endpoints for monitoring platform health and job statistics.

In [ ]:
# Platform Health Check
# Simplified health status for DDH consumption

result = api_call("GET", "/api/platform/health")

if result and result.get("status"):
    print(f"\n📊 Platform Status: {result.get('status', 'unknown')}")
    
    components = result.get("components", {})
    for component, status in components.items():
        icon = "✅" if status == "ok" else "❌"
        print(f"   {icon} {component}: {status}")
    
    activity = result.get("recent_activity", {})
    if activity:
        print(f"\n📈 Recent Activity (24h):")
        print(f"   Jobs: {activity.get('jobs_24h', 0)}")
        print(f"   Success Rate: {activity.get('success_rate', 'N/A')}")

In [ ]:
# Platform Statistics
# Aggregated job statistics over a configurable time window

result = api_call("GET", "/api/platform/stats", params={"hours": 24})

if result:
    print(f"\n📊 Statistics (Last {result.get('time_window_hours', 24)} hours):")
    print(f"   Total Jobs: {result.get('total_jobs', 0)}")
    
    by_status = result.get("by_status", {})
    if by_status:
        print(f"\n   By Status:")
        for status, count in by_status.items():
            print(f"      {status}: {count}")
    
    by_type = result.get("by_data_type", {})
    if by_type:
        print(f"\n   By Data Type:")
        for dtype, count in by_type.items():
            print(f"      {dtype}: {count}")
    
    avg_time = result.get("avg_processing_time_seconds", 0)
    if avg_time:
        print(f"\n   Avg Processing Time: {avg_time:.1f}s")

In [ ]:
# Recent Failures
# Troubleshooting endpoint with sanitized error messages

result = api_call("GET", "/api/platform/failures", params={"hours": 24, "limit": 5})

if result:
    failures = result.get("failures", [])
    total = result.get("total_failures", 0)
    
    print(f"\n📊 Recent Failures: {total} total")
    
    if failures:
        print(f"\nMost Recent:")
        for i, f in enumerate(failures[:5], 1):
            print(f"\n   [{i}] {f.get('job_type', 'unknown')}")
            print(f"       Request: {f.get('request_id', 'N/A')[:16]}...")
            print(f"       Category: {f.get('error_category', 'unknown')}")
            print(f"       Error: {f.get('error_summary', 'N/A')[:60]}...")
    else:
        print("\n   ✅ No failures in the time window")